In [15]:
import pandas as pd
from typing import Dict, Any

# read data
df_policies = pd.read_csv(r"C:\Users\联想\Desktop\3_create_policy_file.csv")
df_mortality = pd.read_csv(r"C:\Users\联想\Downloads\mortality_table.csv")

# Transform motality table into dict
mortality_dict = dict(zip(df_mortality['x'], df_mortality['lx']))



In [19]:
def get_l_x_udd(x: float, mortality_dict: Dict[int, float]) -> float: # UDD Assumption
    x_int = int(x)
    l_x_int = mortality_dict.get(x_int, 0.0)
    l_x_int_plus_1 = mortality_dict.get(x_int + 1, 0.0)
    t = x - x_int
    return l_x_int - t * (l_x_int - l_x_int_plus_1)

In [21]:
def get_t_p_x_udd(t: float, x: float, mortality_dict: Dict[int, float]) -> float:  # living rate
    return get_l_x_udd(x + t, mortality_dict) / get_l_x_udd(x, mortality_dict)

In [37]:
def simulate_liability_for_one_policy(policy_info: dict[str, Any], scenario_info: dict[str, Any], timestep: int) -> float:
    x = policy_info["age"] + timestep/12
    l_x = get_l_x_udd(x=x, mortality_dict=scenario_info["mortality_table"])
    tp_x = get_t_p_x_udd(t=1/12, x=x, mortality_dict=scenario_info["mortality_table"])
    
    # TODO 1: change frequency to monthly, divide premium values and annuity payments by 12.
    monthly_premium = policy_info["premium"] / 12
    monthly_annuity = policy_info["annuity_payments"] / 12
    
    # TODO 2:
    # start_premium_pay_month=0, end_premium_pay_month=45*12,
    # start_receive_payments=45*12, end_receive_payments=100*12
    start_premium_pay_month = policy_info.get("start_premium_pay_month", 0)
    end_premium_pay_month = policy_info.get("end_premium_pay_month", 45*12)
    start_receive_payments = policy_info.get("start_receive_payments", 45*12)
    end_receive_payments = policy_info.get("end_receive_payments", 100*12)
    
    # TODO 3: check where timestep is in relation to premium/receive periods
    cashflow_amount_raw = 0
    if start_premium_pay_month <= timestep < end_premium_pay_month:
        cashflow_amount_raw = monthly_premium  # positive for paying premium
    elif start_receive_payments <= timestep < end_receive_payments:
        cashflow_amount_raw = -monthly_annuity  # negative for receiving annuity
    
    return cashflow_amount_raw * tp_x

In [29]:
# Use the 0th policyholder to do a test
policy_info=df_policies.iloc[0,1:].to_dict()

In [43]:
scenario_info={"mortality_table":mortality_dict}

timestep=12

result= simulate_liability_for_one_policy(policy_info,scenario_info,timestep)

print(f'On timestep {timestep}, Liability cash flow is {result}')

On timestep 12, Liability cash flow is 17408.39835283355
